In [2]:
import os
from dateutil.parser import parse as dateutil_parse
import time
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler as skStandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.cross_validation import train_test_split,cross_val_score
from six.moves import cPickle as pickle

PATH = 'season_1/'
CLEAN_PATH = PATH+'clean/'

from mylib import myStandardScaler,process_order,process_traffic,get_order_group,get_traffic_group,XY_order_traffic, prediction2submit, Search_Model, DISTRICTS

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
cluster_map = pd.read_csv(CLEAN_PATH+'cluster_map.csv',index_col=0)
poi = pd.read_csv(CLEAN_PATH+'poi.csv',index_col=0)
train_order_group = pd.read_pickle(CLEAN_PATH+'train_order_group.pickle')
test_order_group = pd.read_pickle(CLEAN_PATH+'test_order_group.pickle')
train_traffic_group = pd.read_pickle(CLEAN_PATH+'train_traffic_group.pickle')
test_traffic_group = pd.read_pickle(CLEAN_PATH+'test_traffic_group.pickle')
test_target = pd.read_csv(CLEAN_PATH+'test_target.csv',index_col=0,parse_dates=True)

In [11]:
train_slot = pd.Index(sorted(train_order_group.values()[0].index.unique()))
train_slot = pd.Index(filter(lambda x: x%1000 >4,train_slot))
test_slot = test_target['datetimeslot']

In [12]:
now = time.time()

train_XY_group = dict()
for district in DISTRICTS:
    train_XY_group[district] = XY_order_traffic(district,train_order_group,train_traffic_group,train_slot)
test_XY_group = dict()
for district in DISTRICTS:
    test_XY_group[district] = XY_order_traffic(district,test_order_group,test_traffic_group,test_slot)
for district in DISTRICTS:
    scaler = myStandardScaler()
    train_XY_group[district][0] = scaler.fit_transform(train_XY_group[district][0])
    test_XY_group[district][0] = scaler.transform(test_XY_group[district][0])
    
stop = time.time()
print 'Take %02d:%02d:%02d' % ((stop-now)/3600,(stop-now)/60,(stop-now)%60)

Take 00:00:16


In [23]:
grid_params = {'n_estimators': [80] ,'max_depth': np.arange(10, 18), 'min_samples_leaf': [2, 6], 
                     'min_samples_split': [2, 6], 'max_features': ['log2', 'sqrt',None]}
search_models = {district: Search_Model(RandomForestRegressor) for district in DISTRICTS}
test_prediction = dict()
for district, model in search_models.items()[::-1]:
    now = time.time()
    print 'Searching %s...'%district
    model.fit(grid_params,*train_XY_group[district])
    test_prediction[district] = model.predict(test_XY_group[district][0]) - test_XY_group[district][1].fillna(0)
    with open(CLEAN_PATH+'prediction/test_prediction_%s.pickle'%(district),'wb') as f:
        pickle.dump(test_prediction[district],f)
    stop = time.time()
    print 'Take %02d:%02d:%02d' % ((stop-now)/3600,(stop-now)/60,(stop-now)%60)

Searching 2350be163432e42270d2670cb3c02f80...
Best Params: {'warm_start': False, 'oob_score': False, 'n_jobs': 1, 'verbose': 0, 'max_leaf_nodes': None, 'bootstrap': True, 'min_samples_leaf': 6, 'n_estimators': 80, 'min_samples_split': 6, 'min_weight_fraction_leaf': 0.0, 'criterion': 'mse', 'random_state': None, 'max_features': 'log2', 'max_depth': 10}
Fit score: 0.230615025492
The metrics: 0.188342644049
Take 00:05:37
Searching 4b7f6f4e2bf237b6cc58f57142bea5c0...
Best Params: {'warm_start': False, 'oob_score': False, 'n_jobs': 1, 'verbose': 0, 'max_leaf_nodes': None, 'bootstrap': True, 'min_samples_leaf': 6, 'n_estimators': 80, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'criterion': 'mse', 'random_state': None, 'max_features': 'log2', 'max_depth': 13}
Fit score: 0.209694352681
The metrics: 0.16194138973
Take 00:06:11
Searching 82cc4851f9e4faa4e54309f8bb73fd7c...
Best Params: {'warm_start': False, 'oob_score': False, 'n_jobs': 1, 'verbose': 0, 'max_leaf_nodes': None, 'boot

KeyboardInterrupt: 

In [ ]:
submit = prediction2submit(test_prediction, cluster_map)
sublit

In [ ]:
submit.to_csv(PATH+'submit/searchrf_order_traffic.csv',index=None,header=False)